In [1]:
# from sqlalchemy import create_engine

# user = 'postgres'
# password = 'diy3times'
# host = 'localhost'
# database = 'postgres'
# port = 5432
# link = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
# engine = create_engine(link)

In [2]:
from google.cloud import bigquery
import pandas as pd

service_account = "/home/kudadiri/.dbt/dbt-project-data.json"
client = bigquery.Client.from_service_account_json(service_account)

project, dataset, table = "resources-node", "titanic_dataset", "dataset"

table_ref = client.dataset(dataset, project=project).table(table)
table = client.get_table(table_ref)

df = client.list_rows(table).to_dataframe()
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home_dest
0,3,0,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,None,S,None,261.0,None
1,1,0,"Andrews, Mr. Thomas Jr",male,39.0,0,0,112050,0.0,A36,S,None,NaN,"Belfast, NI"
2,1,0,"Chisholm, Mr. Roderick Robert Crispin",male,NaN,0,0,112051,0.0,None,S,None,NaN,"Liverpool, England / Belfast"
3,1,0,"Fry, Mr. Richard",male,NaN,0,0,112058,0.0,B102,S,None,NaN,None
4,1,0,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,B94,S,None,110.0,None


<p>Menggunakan rata-rata umur penumpang yang tersedia, untuk mengisi baris data umur yang kosong<p>

In [3]:
print(df.shape)
print(df.isnull().sum())

age_average = round(df['age'].mean())
df.fillna({'age':age_average}, inplace=True)

df.isnull().sum()

(1309, 14)
pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home_dest     564
dtype: int64


pclass          0
survived        0
name            0
sex             0
age             0
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home_dest     564
dtype: int64

In [4]:
select = df[['ticket', 'cabin', 'boat', 'body', 'home_dest']]
print(select.head(10))
print()
print(select[select['boat'].notnull()].head())
print()
print(select[select['body'].notnull()].head())

   ticket cabin  boat   body                     home_dest
0    3701  None  None  261.0                          None
1  112050   A36  None    NaN                   Belfast, NI
2  112051  None  None    NaN  Liverpool, England / Belfast
3  112058  B102  None    NaN                          None
4  112059   B94  None  110.0                          None
5  112052  None  None    NaN                       Belfast
6   19972  None  None    NaN        Rotterdam, Netherlands
7  239853  None  None    NaN                       Belfast
8  239853  None  None    NaN                       Belfast
9  239854  None  None    NaN                       Belfast

       ticket cabin boat  body       home_dest
823    111163  None    1   NaN    New York, NY
824     11778  C116    1   NaN      Newark, NJ
825  PC 17485   A20    1   NaN  London / Paris
826  PC 17485   E36    1   NaN            None
827     11755   A16    1   NaN  London / Paris

                ticket cabin  boat   body                    home_d

<p>dikarenakan terlalu berisiko untuk menghapus kolom tersebut, diambil keputusan diisi dengan nilai atau keterangan yang menandakan nilai baris kolom tidak diketahui</p>

In [5]:
# print(df[['ticket', 'cabin', 'home_dest']].head(27))
print(select.dtypes)

df.fillna({'cabin': 'Unknown', 'boat': 'No Boat', 'body': 0, 'home_dest': 'Unknown'}, inplace=True)
df.isnull().sum()

ticket        object
cabin         object
boat          object
body         float64
home_dest     object
dtype: object


pclass       0
survived     0
name         0
sex          0
age          0
sibsp        0
parch        0
ticket       0
fare         1
cabin        0
embarked     2
boat         0
body         0
home_dest    0
dtype: int64

In [6]:
# print(df[df['fare'].isnull()], '\n')
# print(df[df['embarked'].isnull()])

df.dropna(inplace=True)
df.isnull().sum()

pclass       0
survived     0
name         0
sex          0
age          0
sibsp        0
parch        0
ticket       0
fare         0
cabin        0
embarked     0
boat         0
body         0
home_dest    0
dtype: int64

In [11]:
# df.to_sql(
#     "../models/titanic_data.sql",
#     con=engine,
#     if_exists='replace',
#     index=False)

df.to_csv("../seeds/titanic_data.csv", index=False)